In [ ]:
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
Path = mpath.Path
import matplotlib.patches as mpatches
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import numpy.typing as npt
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.interpolate import interp1d

from typing import cast, Callable
from __future__ import annotations

In [ ]:
width = 200
height = 200
depth = 20
cellCount = 10

# Synthesize 2d data

In [ ]:
Point = npt.NDArray[np.float64]

points = np.random.randint([0, 0], [[width, height]], size=(cellCount, 2))
v = Voronoi(points)
print(v.points)
fig = voronoi_plot_2d(v, show_points=True, show_vertices=True)
plt.show()

In [ ]:
def in_polygon(point: Point, edge_vertices: list[Point]) -> bool:
  n = len(edge_vertices)
  sign = None

  for i in range(n):
    p1 = edge_vertices[i]
    p2 = edge_vertices[(i + 1) % n]
    
    v1 = p2 - p1
    v2 = point - p1
    s = np.sign(np.cross(v1, v2))
    if sign is None:
      sign = s
    elif s == 0:
      return False
    elif sign != s:
      return False
  return True

In [ ]:
def random_float(lower: float = 0, upper: float = 1) -> float:
  return np.random.random() * (upper - lower)  + lower

In [ ]:
def vector_angle(v1: Point, v2: Point) -> float:
  cross = np.cross(v1, v2)
  dot = np.dot(v1, v2)
  return np.arctan2(cross, dot)

In [ ]:
def rotate_point(origin: Point, angle: float, points: npt.NDArray[np.float64]) -> Point:
  return np.array([
    np.cos(angle) * (points[:, 0] - origin[0]) - np.sin(angle) * (points[:, 1] - origin[1]) + origin[0],
    np.sin(angle) * (points[:, 0] - origin[0]) + np.cos(angle) * (points[:, 1] - origin[1]) + origin[1]
  ]).T

In [ ]:
def bezier_quadratic(p0: Point, p1: Point, p2: Point) -> Callable[[npt.NDArray[np.float64]], npt.NDArray[np.float64]]:
  p0 = p0.reshape(1, -1)
  p1 = p1.reshape(1, -1)
  p2 = p2.reshape(1, -1)

  def interpolate(t: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    t = t.reshape(-1, 1)
    return p1 + (1-t)**2 * (p0 - p1) + t**2 * (p2 - p1)
  return interpolate

In [ ]:
class Viewport:
  def __init__(self, origin: Point, width: int, height: int):
    assert width > 0 and height > 0
    self.origin = origin
    self.width = width
    self.height = height

  def contains(self, point: Point) -> bool:
    return self.origin[0] <= point[0] <= self.origin[0] + self.width \
      and self.origin[1] <= point[1] <= self.origin[1] + self.height

In [ ]:
class Line:
  def __init__(self, center_point: Point, start_point: Point):
    self.center_point = center_point
    self.start_point = start_point
  
  def get_intersection(self, other: Line) -> Point:
    p = self.start_point
    px, py = p
    vx, vy = self.get_direction()

    q = other.start_point if isinstance(other, OpenSegment) else other.end_point
    qx, qy = q
    w = other.get_direction()
    wx, wy = w

    multiplier = (vy * (qx - px) + py * vx - qy * vx) / (vx * wy - vy * wx)
    return q + multiplier * w

In [ ]:
class LineSegment(Line):
  def __init__(self, center_point: Point, start_point: Point, end_point: Point):
    super().__init__(center_point, start_point)
    self.end_point = end_point

    end_point = cast(Point, end_point)
    diff = start_point - end_point
    normal = np.array([-diff[1], diff[0]])

    # invert vector if it's facing outwards
    vec_to_center = center_point - start_point
    angle = np.arccos(np.dot(vec_to_center, normal) / (np.linalg.norm(vec_to_center) * np.linalg.norm(normal)))
    if angle > np.pi / 2:
      normal *= [-1, -1]
    self.normal = normal / np.linalg.norm(normal)
      
  # return whether the provided point is one of the segment points
  def has_segment_point(self, point: Point) -> bool:
    return (self.start_point == point).all() or (self.end_point == point).all()

  def get_end_point(self, viewport: Viewport) -> Point:
    return self.end_point

  def change_position(self, change: float):
    self.start_point += self.normal * change
    self.end_point += self.normal * change

  def get_direction(self):
    diff = self.end_point - self.start_point
    return diff / np.linalg.norm(diff)

  def create_kink_points(self, config: EdgeDecompositionConfig, viewport: Viewport) -> list[Point]:
    direction = self.end_point - self.start_point
    dist = np.linalg.norm(direction)

    i = 0
    kink_locations: list[float] = []
    while i < dist - config.min_distance:
      kink_locations.append(i / dist)
      i += np.random.random() * (config.max_distance - config.min_distance) + config.min_distance
    
    if len(kink_locations) <= 1:
      return []
    del kink_locations[0]

    return [self.start_point + direction * kink + self.normal * random_float(-config.normal_shift, config.normal_shift) for kink in kink_locations]

  def decompose(self, config: EdgeDecompositionConfig, viewport: Viewport) -> list[LineSegment]:
    new_edges = []
    kink_points = self.create_kink_points(config, viewport)
    if not kink_points:
      return new_edges

    new_edges.append(LineSegment(self.center_point, self.start_point.copy(), kink_points[0]))
    for i in range(len(kink_points) - 1):
      start = kink_points[i]
      end = kink_points[i + 1]
      new_edges.append(LineSegment(self.center_point, start.copy(), end))
    new_edges.append(LineSegment(self.center_point, kink_points[-1].copy(), self.end_point.copy()))

    return new_edges

  def __str__(self) -> str:
    return f"LineSegment(center_point={self.center_point}, start_point={self.start_point}, end_point={self.end_point}"

In [ ]:
class OpenSegment(Line):
  def __init__(self, center_point: Point, start_point: Point, normal: Point, direction: Point):
    super().__init__(center_point, start_point)
    self.direction = direction
    self.normal = normal

  @classmethod
  def from_voronoi(cls, v: Voronoi, point_indices: tuple[int, int], center_point: Point, start_point: Point, normal: Point) -> OpenSegment:
    n = np.array([-normal[1], normal[0]])

    midpoint = v.points[list(point_indices)].mean(axis=0)
    center = v.points.mean(axis=0)
    direction = np.sign(np.dot(midpoint - center, n)) * n
    direction /= np.linalg.norm(direction)

    # invert vector if it's facing outwards
    vec_to_center = center_point - start_point
    angle = np.arccos(np.dot(vec_to_center, normal) / (np.linalg.norm(vec_to_center) * np.linalg.norm(normal)))
    if angle > np.pi / 2:
      normal *= [-1, -1]
    normal = normal / np.linalg.norm(normal)

    return cls(center_point, start_point, normal, direction)

  @classmethod
  def from_open_segment(cls, open_segment: OpenSegment, start_point: Point) -> OpenSegment:
    return cls(open_segment.center_point, start_point, open_segment.normal, open_segment.direction)
      
  def has_segment_point(self, point: Point) -> bool:
    return (self.start_point == point).all()

  def change_position(self, change: float):
    self.start_point += self.normal * change

  def get_direction(self):
    return self.direction

  def get_end_point(self, viewport: Viewport) -> Point:
    if not viewport.contains(self.start_point):
      return self.start_point
    border_x = viewport.origin[0] if self.direction[0] < 0 else viewport.origin[0] + viewport.width

    multiplier = (border_x - self.start_point[0]) / self.direction[0]
    end_point = self.start_point + multiplier * self.direction
    if viewport.contains(end_point):
      return end_point

    border_y = viewport.origin[1] if self.direction[1] < 0 else viewport.origin[1] + viewport.height
    multiplier = (border_y - self.start_point[1]) / self.direction[1]
    end_point = self.start_point + multiplier * self.direction
    return end_point

  def create_kink_points(self, config: EdgeDecompositionConfig, viewport: Viewport) -> list[Point]:
    end_point = self.get_end_point(viewport)
    dist = np.linalg.norm(self.start_point - end_point)
    direction = self.direction * dist

    i = 0
    kink_locations: list[float] = []
    while i < dist - config.min_distance:
      kink_locations.append(i / dist)
      i += np.random.random() * (config.max_distance - config.min_distance) + config.min_distance
    
    if len(kink_locations) <= 1:
      return []
    del kink_locations[0]

    return [self.start_point + direction * kink + self.normal * random_float(-config.normal_shift, config.normal_shift) for kink in kink_locations]

  def decompose(self, config: EdgeDecompositionConfig, viewport: Viewport) -> list[Line]:
    new_edges = []
    end_point = self.get_end_point(viewport)
    kink_points = self.create_kink_points(config, viewport)
    if not kink_points:
      return new_edges

    new_edges.append(LineSegment(self.center_point, self.start_point.copy(), kink_points[0]))
    for i in range(len(kink_points) - 1):
      start = kink_points[i]
      end = kink_points[i + 1]
      new_edges.append(LineSegment(self.center_point, start.copy(), end))
    new_edges.append(LineSegment(self.center_point, kink_points[-1].copy(), end_point.copy()))
    new_edges.append(OpenSegment.from_open_segment(self, end_point))

    return new_edges

  def __str__(self) -> str:
    return f"OpenSegment(center_point={self.center_point}, start_point={self.start_point}, normal={self.normal}, direction={self.direction}"

In [ ]:
class Polygon:
  edges: list[Line]
  v: Voronoi

  def __init__(self, v: Voronoi, point: int):
    self.edges = []
    pointCount = len(v.ridge_points)
    center_point = v.points[point]
    rp = np.concatenate((v.ridge_points, np.arange(pointCount).reshape((pointCount, 1))), axis=1)
    ridges = rp[np.logical_or.reduce(rp[:, :-1] == point, 1)]
    point_ridges =  ridges[:, :-1]
    ridge_indices = ridges[:, -1].flatten()

    vertex_ridges = [vertex for (i, vertex) in enumerate(v.ridge_vertices) if i in ridge_indices]
    ridges_to_construct = []
    for i, vertex in enumerate(vertex_ridges):
      assert len(vertex) == 2
      assert vertex[0] != -1 or vertex[1] != -1

      if vertex[0] != -1 and vertex[1] != -1:
        self.edges.append(LineSegment(center_point, v.vertices[vertex[0]].copy(), v.vertices[vertex[1]].copy()))
      else:
        ridges_to_construct.append(i)
    
    for i in ridges_to_construct:
      vertices = vertex_ridges[i]
      start_point = v.vertices[vertices[0] if vertices[0] != -1 else vertices[1]]
      normal = v.points[point_ridges[i][0]] - v.points[point_ridges[i][1]]
      self.edges.append(OpenSegment.from_voronoi(v, (point_ridges[i][0], point_ridges[i][1]), center_point, start_point.copy(), normal))

    # sort lines s.t. adjacent lines are adjacent in the list
    for i in range(len(self.edges) - 2, 0, -1):
      intersection = self.edges[i + 1].start_point
      neighbor: Line = next(filter(lambda line: line.has_segment_point(intersection), self.edges))
      neighbor_index = self.edges.index(neighbor)
      self.edges[neighbor_index], self.edges[i] = self.edges[i], self.edges[neighbor_index]

      if (neighbor.start_point == intersection).all():
        neighbor.start_point, neighbor.end_point = neighbor.end_point, neighbor.start_point
    
    if len(self.edges) >= 2 and isinstance(self.edges[0], LineSegment) and (self.edges[1].start_point == self.edges[0].start_point).all():
        self.edges[0].start_point, self.edges[0].end_point = self.edges[0].end_point, self.edges[0].start_point

  def change_size(self, change: float):
    for edge in self.edges:
      edge.change_position(change)
    for i in range(len(self.edges) - 1, -1, -1):
      line1 = self.edges[i]
      line2 = self.edges[i - 1]
      if isinstance(line1, OpenSegment) and isinstance(line2, OpenSegment) and i == 0:
        continue
      p = line1.start_point
      q = line2.start_point if isinstance(self.edges[i - 1], OpenSegment) else line2.end_point

      intersection = line1.get_intersection(line2)
      p[0] = intersection[0]
      p[1] = intersection[1]
      q[0] = intersection[0]
      q[1] = intersection[1]
    self.prune_edges()

  # check if edges can be deleted (e.g. after resizing)
  def prune_edges(self):
    for i in range(len(self.edges) - 2, 0, -1):
      line = self.edges[i]
      line_after = self.edges[i + 1]
      line_before = self.edges[i - 1]

      try:
        intersection = line_after.get_intersection(line_before)
      except ZeroDivisionError:
        continue

      if not in_polygon(intersection, [line.start_point, line.end_point, line.center_point]):
        continue

      line_after.start_point = intersection
      line_before_point = line_before.end_point if isinstance(line_before, LineSegment) else line_before.start_point
      line_before_point[0] = intersection[0]
      line_before_point[1] = intersection[1]

      del self.edges[i]

  def decompose_edges(self, config: EdgeDecompositionConfig, viewport: Viewport):
    for i in range(len(self.edges) - 1, -1, -1):
      new_edges = self.edges[i].decompose(config, viewport)
      if not new_edges:
        continue
      self.edges[i:i+1] = new_edges

In [ ]:
class PolygonCollection:
  polygons: list[Polygon]
  viewport: Viewport

  def __init__(self, v: Voronoi, edge_decomposition_config: EdgeDecompositionConfig, pixel_per_interpolation: int):
    self.polygons = [Polygon(v, i) for i in range(len(v.points))]
    stretch = v.points.ptp(0)
    minimum = v.points.min(0)

    self.viewport = Viewport(minimum - stretch * 0.1, stretch[0] * 1.2, stretch[1] * 1.2)
    self.pixel_per_interpolation = 5
    self.edge_decomposition_config = edge_decomposition_config

  def interpolate_line(self, line: Line, start: Point, end: Point) -> list[list[Point]]:
    dist = np.linalg.norm(start - end)

    direction = line.get_direction() * dist
    x_axis = np.array([1, 0])
    angle = vector_angle(direction, x_axis)

    rotated_end = rotate_point(start, angle, np.array([end]))[0]
    temp_line = LineSegment(line.center_point, start, rotated_end)
    kink_points = np.array(temp_line.create_kink_points(self.edge_decomposition_config, self.viewport))

    if len(kink_points) == 0:
      return [[[start[0], start[1]], [end[0], end[1]]]]

    x = np.array([start[0]] + [point[0] for point in kink_points] + [rotated_end[0]])
    y = np.array([start[1]] + [point[1] for point in kink_points] + [rotated_end[1]])
    interpolation = interp1d(x, y, kind='quadratic')

    n_samples = int(np.ceil(dist / self.pixel_per_interpolation))
    # new_x = np.array([start[0] + multiplier * direction[0] for multiplier in np.linspace(0, 1, n_samples, endpoint=False)])
    rotated_x = np.array([start[0] + multiplier * dist for multiplier in np.linspace(0, 1, n_samples, endpoint=False)])
    points = np.concatenate((rotated_x.reshape((-1, 1)), interpolation(rotated_x).reshape(-1, 1)), axis=1)
    points = rotate_point(start, -angle, points)
    lines = []

    for i, point in enumerate(points):
      if i == len(points) - 1:
        lines.append([[point[0], point[1]], [end[0], end[1]]])
      else:
        lines.append([[point[0], point[1]], [points[i+1, 0], points[i+1, 1]]])
    return lines

  def interpolate_edge(self, start: Point, end: Point, intersection: Point) -> list[Point]:
    dist = np.linalg.norm(start - intersection) + np.linalg.norm(intersection - end)
    n_samples = int(np.ceil(dist / self.pixel_per_interpolation))
    t = np.linspace(0, 1, n_samples)
    interpolated = bezier_quadratic(start, intersection, end)(t)
    return [[[x[0], x[1]], [interpolated[i + 1, 0], interpolated[i + 1, 1]]] for i, x in enumerate(interpolated) if i != len(interpolated) - 1]

  def plot(self):
    lines = []
    fig, ax = plt.subplots()
    for j, p in enumerate(self.polygons):
      first_edge_start = None
      neighbor_start = None
      printed = 0

      for i, line in enumerate(p.edges):
        line_length = np.linalg.norm(line.start_point - line.get_end_point(self.viewport))
        smoothing_length = min(line_length / 2, random_float(self.edge_decomposition_config.min_corner_smoothing, self.edge_decomposition_config.max_corner_smoothing))
        if isinstance(line, OpenSegment):
          current_end = line.start_point + line.get_direction() * smoothing_length
        else:
          current_end = line.end_point - line.get_direction() * smoothing_length

        if neighbor_start is None:
          smoothing_length = min(line_length / 2, random_float(self.edge_decomposition_config.min_corner_smoothing, self.edge_decomposition_config.max_corner_smoothing))
          current_start = line.start_point + line.get_direction() * smoothing_length
          first_edge_start = current_start
        else:
          current_start = neighbor_start

        if i == len(p.edges) - 1:
          neighbor_start = first_edge_start
          next_line = p.edges[0]
        else:
          next_line = p.edges[i+1]
          next_line_length = np.linalg.norm(next_line.start_point - next_line.get_end_point(self.viewport))
          next_smoothing_length = min(next_line_length / 2, random_float(self.edge_decomposition_config.min_corner_smoothing, self.edge_decomposition_config.max_corner_smoothing))
          neighbor_start = next_line.start_point + next_line.get_direction() * next_smoothing_length

        if isinstance(line, OpenSegment) or np.linalg.norm(current_start - current_end) > 1e-10:
          lines.extend(self.interpolate_line(line, current_start, current_end if isinstance(line, LineSegment) else line.get_end_point(self.viewport)))

        if isinstance(line, LineSegment):
          lines.extend(self.interpolate_edge(current_end, neighbor_start, next_line.start_point))
        elif i != len(p.edges) - 1:
          lines.extend(self.interpolate_edge(current_start, neighbor_start, next_line.start_point))
        printed -= 1

    ax.add_collection(LineCollection(lines))
    ax.set_xlim(self.viewport.origin[0], self.viewport.origin[0] + self.viewport.width)
    ax.set_ylim(self.viewport.origin[1], self.viewport.origin[1] + self.viewport.height)
    plt.show()

  def decompose_edges(self, config: EdgeDecompositionConfig, viewport: Viewport):
    for polygon in self.polygons:
      polygon.decompose_edges(config, viewport)

In [ ]:
class EdgeDecompositionConfig:
  def __init__(self, normal_shift: float, min_distance: float, max_distance: float, min_corner_smoothing: float, max_corner_smoothing: float):
    self.normal_shift = normal_shift # how far the new points are maximally away from the original line
    
    # the distance between kink points
    self.min_distance = min_distance
    self.max_distance = max_distance

    # how far to move in start/end of the lines for smoothing the transition between them
    self.min_corner_smoothing = min_corner_smoothing
    self.max_corner_smoothing = max_corner_smoothing

In [ ]:
viewport = Viewport(np.array([0, 0]), 200, 200)
edge_decomposition_config = EdgeDecompositionConfig(3, 10, 20, 5, 20)

In [ ]:
# np.random.set_state(test_state)
col = PolygonCollection(v, edge_decomposition_config, 2)
for p in col.polygons:
  p.change_size(3)
col.plot()